Import libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

Get Newyork neighbourhood data

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
newyork_data    

Data downloaded!


{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

In [3]:
neighborhoods_data = newyork_data['features']
#Transform data into to dataframe
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
for data in neighborhoods_data:
    borough = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
print(manhattan_data.head())
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan


     Borough        Neighborhood   Latitude  Longitude
0  Manhattan         Marble Hill  40.876551 -73.910660
1  Manhattan           Chinatown  40.715618 -73.994279
2  Manhattan  Washington Heights  40.851903 -73.936900
3  Manhattan              Inwood  40.867684 -73.921210
4  Manhattan    Hamilton Heights  40.823604 -73.949688
The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


Explore Marble hill

In [4]:
CLIENT_ID = 'ROZBATJPUJS2EJ55CETLI23AFTD3JLESDQNMD4SA1TXVX51C' # your Foursquare ID
CLIENT_SECRET = 'L4U0MC1QO0BLQREMSCGT0MPWPESGMPY320VCFMTS5HALUGMH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))
#get top 100 venues in MArble hill
radius = 1000 # define radius
LIMIT  =100
 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()
results
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

Latitude and longitude values of Marble Hill are 40.87655077879964, -73.91065965862981.


,name,categories,lat,lng
0,Bikram Yoga,Yoga Studio,40.876844,-73.906204
1,Arturo's,Pizza Place,40.874412,-73.910271
2,Tibbett Diner,Diner,40.880404,-73.908937
3,Sam's Pizza,Pizza Place,40.879435,-73.905859
4,Starbucks,Coffee Shop,40.877531,-73.905582


In [25]:
nearby_venues['categories'].value_counts().head(10)
top_venues = []
#Add this to the dataframe to compare with the Toronto Neighborhoods
df1 = nearby_venues['categories'].value_counts().head(10).rename_axis('Category').reset_index(name='Count')
df1['Neighborhood'] = 'Marble Hill'
df_compare = df1
df_compare

,Category,Count,Neighborhood
0,Park,6,Marble Hill
1,Spanish Restaurant,3,Marble Hill
2,Sandwich Place,3,Marble Hill
3,Pizza Place,3,Marble Hill
4,Donut Shop,3,Marble Hill
5,Café,3,Marble Hill
6,Mexican Restaurant,3,Marble Hill
7,Supermarket,3,Marble Hill
8,Pharmacy,2,Marble Hill
9,Coffee Shop,2,Marble Hill


Now explore the Toronto Boroughs

In [5]:

import requests
import pandas as pd
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
My_table = soup.find('table',{'class':'wikitable sortable'})
Neighbourhood = []
#read the table into an array
for row in My_table.find_all('tr'):
    col = row.findAll('td')
    if len(col) > 0:
        postcode = col[0].string
        county = col[1].string
        borough = col[2].string
        Neighbourhood.append([t.text.strip() for t in row.find_all('td')])
        
df = pd.DataFrame([])
#populate the dataframe from array
df = pd.DataFrame(Neighbourhood, columns=['PostalCode', 'Borough', 'Neighbourhood'])
 # Get names of indexes for which column 'Borough is Not assigned'
indexNames = df[ df['Borough'] == 'Not assigned' ].index
    
# Delete these row indexes from dataFrame
df.drop(indexNames , inplace=True)
df.head() 
df['Neighbourhood'] = df.groupby(['PostalCode', 'Borough'])['Neighbourhood'].transform(lambda x: ', '.join(set(x.dropna())))
df.drop_duplicates()
df.loc[df['Neighbourhood'] =='Not assigned','Neighbourhood']=df['Borough']
#Get coordinates
# Load the Pandas libraries with alias 'pd' 
# Read data from file 
data = pd.read_csv("http://cocl.us/Geospatial_data/Geospatial_coordinates.csv") 
# change the column name of the first column
data.columns = ['PostalCode','Latitude','Logitude']
#Merge the data of two dataframes
df_final = pd.merge(df, data, on='PostalCode')

In [33]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="CA_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [6]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
#Slice the dataframe to get only toronto borogh
df_toronto = df_final[df_final['Borough'].str.contains (u'Toronto')].reset_index(drop=True)
print(df_toronto.head())
# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Logitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto


  PostalCode           Borough              Neighbourhood   Latitude  \
0        M5A  Downtown Toronto  Harbourfront, Regent Park  43.654260   
1        M5A  Downtown Toronto  Harbourfront, Regent Park  43.654260   
2        M5B  Downtown Toronto   Garden District, Ryerson  43.657162   
3        M5B  Downtown Toronto   Garden District, Ryerson  43.657162   
4        M5C  Downtown Toronto             St. James Town  43.651494   

    Logitude  
0 -79.360636  
1 -79.360636  
2 -79.378937  
3 -79.378937  
4 -79.375418  


In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        nearby_venues = json_normalize(venues) # flatten JSON
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [12]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighbourhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Logitude']
                                  )
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront, Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront, Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront, Regent Park",43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,"Harbourfront, Regent Park",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,"Harbourfront, Regent Park",43.65426,-79.360636,The Distillery Historic District,43.650244,-79.359323,Historic Site


In [58]:
toronto_venues_sum = toronto_venues.groupby(["Neighborhood", "Venue Category"]).size().reset_index(name="Count")
#df1['Neighborhood'] = 'Marble Hill'
#df_compare = df1
toronto_venues_sum.head(10)
print(df1)
indexNames = toronto_venues_sum[ (toronto_venues_sum['Venue Category'] != 'Pharmacy') & (toronto_venues_sum['Venue Category'] != 'Park') & (toronto_venues_sum['Venue Category'] != 'Supermarket' ) &  (toronto_venues_sum['Venue Category'] != 'Mexican Restaurant' )  ].index
toronto_venues_sum.drop(indexNames , inplace=True)
indexNames1 = df1[ (df1['Venue Category'] != 'Pharmacy') & (df1['Venue Category'] != 'Park') & (df1['Venue Category'] != 'Supermarket' ) &  (df1['Venue Category'] != 'Mexican Restaurant' )  ].index
df1.drop(indexNames1 , inplace=True)
df_compare= df1
df_compare.columns=['Venue Category','Count', 'Neighborhood']        
print(df_compare)
print(toronto_venues_sum.shape)
print(df_compare.shape)
df_final=pd.concat([df_compare, toronto_venues_sum])

print(df_final.shape)
df_final


       Venue Category  Count Neighborhood
0                Park      6  Marble Hill
1  Spanish Restaurant      3  Marble Hill
2      Sandwich Place      3  Marble Hill
3         Pizza Place      3  Marble Hill
4          Donut Shop      3  Marble Hill
5                Café      3  Marble Hill
6  Mexican Restaurant      3  Marble Hill
7         Supermarket      3  Marble Hill
8            Pharmacy      2  Marble Hill
9         Coffee Shop      2  Marble Hill
       Venue Category  Count Neighborhood
0                Park      6  Marble Hill
6  Mexican Restaurant      3  Marble Hill
7         Supermarket      3  Marble Hill
8            Pharmacy      2  Marble Hill
(76, 3)
(4, 3)
(80, 3)


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:15: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,Count,Neighborhood,Venue Category
0,6,Marble Hill,Park
6,3,Marble Hill,Mexican Restaurant
7,3,Marble Hill,Supermarket
8,2,Marble Hill,Pharmacy
46,3,Berczy Park,Park
55,1,Berczy Park,Supermarket
85,4,Business Reply Mail Processing Centre 969 Eastern,Park
136,3,Central Bay Street,Park
149,1,Central Bay Street,Supermarket
195,2,Christie,Mexican Restaurant


From the study of Marble Hill meighborhood, it predominantly has Mexican restaurants, parks and supermarkets, Pharmacies
Lets concentrate on only these parameter and  see which neighborhoods in Toronto has all these

In [71]:
df = df_final.pivot(index ='Neighborhood', columns='Venue Category')
df = df.dropna()
df

Count                           
Venue Category               Mexican Restaurant  Park Pharmacy Supermarket
Neighborhood                                                              
Davisville                                  2.0   1.0      1.0         1.0
Davisville North                            2.0   1.0      3.0         1.0
Dovercourt Village, Dufferin                2.0  12.0      4.0         4.0
Marble Hill                                 3.0   6.0      2.0         3.0
The Beaches                                 1.0   2.0      1.0         1.0

From the exploratory analysis it is seen that below neighborhoods in Toronto have similar venues (Mexican Restaurants, Park, Pharmacy and supermarket) like Marble hill neighborhood
Davisville
Davisville North
Dovercourt Village, Dufferin
The Beaches